Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [4]:
from sqlalchemy import *
import datetime
import mediacloud

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [7]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('results', INT)
)
metadata.create_all(engine) # and create the tables in the database

2016-12-06 17:55:34,424 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-06 17:55:34,424 INFO sqlalchemy.engine.base.Engine ()
2016-12-06 17:55:34,427 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-06 17:55:34,427 INFO sqlalchemy.engine.base.Engine ()
2016-12-06 17:55:34,430 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2016-12-06 17:55:34,430 INFO sqlalchemy.engine.base.Engine ()
2016-12-06 17:55:34,433 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	timestamp DATETIME, 
	results INTEGER, 
	PRIMARY KEY (id)
)


2016-12-06 17:55:34,434 INFO sqlalchemy.engine.base.Engine ()
2016-12-06 17:55:34,436 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [8]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, results) VALUES (:id, :keywords, :timestamp, :results)'

In [9]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt)

'INSERT INTO queries (keywords, timestamp) VALUES (:keywords, :timestamp)'

In [10]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-06 17:56:47,085 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2016-12-06 17:56:47,085 INFO sqlalchemy.engine.base.Engine ('puppies', '2016-12-06 17:56:47.085000')
2016-12-06 17:56:47,088 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

In [11]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-06 18:00:47,338 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2016-12-06 18:00:47,338 INFO sqlalchemy.engine.base.Engine ('kittens', '2016-12-06 18:00:47.338000')
2016-12-06 18:00:47,339 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [12]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

2016-12-06 18:01:00,446 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.results 
FROM queries
2016-12-06 18:01:00,447 INFO sqlalchemy.engine.base.Engine ()
(1, u'puppies', datetime.datetime(2016, 12, 6, 17, 56, 47, 85000), None)
(2, u'kittens', datetime.datetime(2016, 12, 6, 18, 0, 47, 338000), None)


In [ ]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [ ]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [ ]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [ ]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [ ]:
query = Query(keywords="iguana")
query.keywords

In [ ]:
my_session.add(query)
my_session.commit()
query.id

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q

In [35]:
import mediacloud, datetime
f = open('private_key.txt', 'r')
privatekey = f.read()
mc = mediacloud.api.MediaCloud(privatekey)
keyword = 'Clinton'
res_c = mc.sentenceCount(keyword, solr_filter=[mc.publish_date_query( datetime.date( 2016, 1, 9), datetime.date( 2016, 1, 10) ), 'media_sets_id:1' ])
insert_stmt = queries.insert().values(keywords=keyword, results=res_c['count'])
print str(insert_stmt)
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

keyword = 'Trump'
res_t = mc.sentenceCount(keyword, solr_filter=[mc.publish_date_query( datetime.date( 2016, 1, 9), datetime.date( 2016, 1, 10) ), 'media_sets_id:1' ])
insert_stmt = queries.insert().values(keywords=keyword, results=res_t['count'])
print str(insert_stmt)
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned




INSERT INTO queries (keywords, timestamp, results) VALUES (:keywords, :timestamp, :results)
2016-12-06 18:12:10,476 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp, results) VALUES (?, ?, ?)
2016-12-06 18:12:10,477 INFO sqlalchemy.engine.base.Engine ('Clinton', '2016-12-06 18:12:10.476000', 16)
2016-12-06 18:12:10,480 INFO sqlalchemy.engine.base.Engine COMMIT
INSERT INTO queries (keywords, timestamp, results) VALUES (:keywords, :timestamp, :results)
2016-12-06 18:12:10,733 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp, results) VALUES (?, ?, ?)
2016-12-06 18:12:10,733 INFO sqlalchemy.engine.base.Engine ('Trump', '2016-12-06 18:12:10.733000', 57)
2016-12-06 18:12:10,736 INFO sqlalchemy.engine.base.Engine COMMIT


[22]

In [36]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row


2016-12-06 18:12:11,457 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.results 
FROM queries
2016-12-06 18:12:11,460 INFO sqlalchemy.engine.base.Engine ()
(1, u'puppies', datetime.datetime(2016, 12, 6, 17, 56, 47, 85000), None)
(2, u'kittens', datetime.datetime(2016, 12, 6, 18, 0, 47, 338000), None)
(3, u'keyword', datetime.datetime(2016, 12, 6, 18, 7, 45, 274000), 16)
(4, u'keyword', datetime.datetime(2016, 12, 6, 18, 7, 45, 570000), 57)
(5, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 7, 819000), 16)
(6, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 8, 130000), 57)
(7, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 20, 339000), 1)
(8, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 20, 591000), 0)
(9, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 29, 808000), 11)
(10, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 30, 140000), 0)
(11, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 36, 688000), 1)
(12, u'keywo

In [69]:
threshold = 1
select_stmt = select([queries]).where(queries.c.results>threshold)
print 'List of queries with results greater than',threshold,':\n'
for row in db_conn.execute(select_stmt):
    print row

List of queries with results greater than 1 :

2016-12-06 18:21:33,266 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.results 
FROM queries 
WHERE queries.results > ?
2016-12-06 18:21:33,267 INFO sqlalchemy.engine.base.Engine (1,)
(3, u'keyword', datetime.datetime(2016, 12, 6, 18, 7, 45, 274000), 16)
(4, u'keyword', datetime.datetime(2016, 12, 6, 18, 7, 45, 570000), 57)
(5, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 7, 819000), 16)
(6, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 8, 130000), 57)
(9, u'keyword', datetime.datetime(2016, 12, 6, 18, 8, 29, 808000), 11)
(15, u'(Clinton)', datetime.datetime(2016, 12, 6, 18, 11, 36, 441000), 16)
(16, u'(Trump)', datetime.datetime(2016, 12, 6, 18, 11, 36, 680000), 57)
(17, u'Clinton', datetime.datetime(2016, 12, 6, 18, 12, 9, 355000), 16)
(18, u'Trump', datetime.datetime(2016, 12, 6, 18, 12, 9, 625000), 57)
(19, u'Clinton', datetime.datetime(2016, 12, 6, 18, 12, 9, 865000), 16)
(2

In [54]:
term = 'trum%'
select_stmt = select([queries]).where(queries.c.keywords.like(term))
for row in db_conn.execute(select_stmt):
    print row

2016-12-06 18:17:43,782 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.results 
FROM queries 
WHERE queries.keywords LIKE ?
2016-12-06 18:17:43,782 INFO sqlalchemy.engine.base.Engine ('trum%',)
(18, u'Trump', datetime.datetime(2016, 12, 6, 18, 12, 9, 625000), 57)
(20, u'Trump', datetime.datetime(2016, 12, 6, 18, 12, 10, 179000), 57)
(22, u'Trump', datetime.datetime(2016, 12, 6, 18, 12, 10, 733000), 57)


In [53]:
term = 'trum%'
select_stmt = select([queries]).where(queries.c.keywords.like(term))
sum = 0
for row in db_conn.execute(select_stmt):
    sum += row[3]
print 'Sum of all queries that contain the term ***',term,'*** is',sum

2016-12-06 18:17:24,520 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.results 
FROM queries 
WHERE queries.keywords LIKE ?
2016-12-06 18:17:24,523 INFO sqlalchemy.engine.base.Engine ('trum%',)
Sum of all queries that contain the term *** trum% *** is 171
